In [16]:
import numpy as np
import scipy.linalg as la

In [106]:
m , n = 5, 4
A = np.random.randn(m,n)
A

array([[-0.28334707, -0.13120762, -0.20774252,  1.16265585],
       [-1.25202378, -0.59953921,  0.46867528, -0.06123326],
       [-0.16205225, -1.93986676, -0.54373147, -1.60366592],
       [-0.39449985, -1.66466674, -0.6000887 ,  0.99746121],
       [ 0.58503363,  0.3773792 ,  1.75879817, -0.49982664]])

In [ ]:
A[0] = 0 # causes issue
A[:,0] = 0 # causes issue
A[0,0] = 0 # works fine

In [111]:
def LEUP_fact(M):
    m, n = M.shape
    i, j = 0,0
    E = np.array(M, copy = True)
    L = np.eye(m)
    U = np.eye(n)
    P = np.eye(n)
    while i < m and j < n:
        # find column to swap with
        j2 = j
        while j2 < n and E[i,j2] == 0:
            j2 += 1
        if j2 == n:
            # increment row
            print(i)
            i = i+1
            continue
        # else, we have a pivot
        # swap columns of E, U
        print(j,j2)
        E[i:,[j, j2]] = E[i:,[j2,j]]
        U[:,[j,j2]] = U[:,[j2,j]]
        
        # swap rows of U and P
        U[[j, j2]] = U[[j2, j]] 
        P[[j, j2]]  = P[[j2, j]] 
        
        # now do a schur complement
        Aij = E[i,j].reshape(-1,1)
        A2j = E[(i+1):,j].reshape(-1,1)
        Ai2 = E[i,(j+1):].reshape(1,-1)
        A22 = E[(i+1):,(j+1):]
        
        
        U2 = np.eye(n)
        try:
            U2[i,(j+1):] = (la.inv(Aij)) @ Ai2 
        except:
            print('error with U at {},{}'.format(i,j))
        U = U2 @ U
        
        L2 = np.eye(m)
        try:
            L2[(i+1):,j] = (A2j @ (la.inv(Aij))).flatten()
            L = L @ L2
        except:
            print('error with L')
        
        # form schur complement and apply
        try:
            S = A22 - (A2j @ (1/(Aij)) @ (Ai2))
            E[(i+1):,(j+1):] = S
            E[i,(j+1):] = 0
            E[(i+1):,j] = 0
        except:
            print("error with E")

        j = j+1
        i = i+1
        
        
    return L, E, U, P.T
        

TODO:
* Debug issues above
* Rational arithmetic (try Python libaries first),
* Shape commutation algorithm (Prop 7)
* Quiver algorithm (Alg 5)

Last resort:
https://caam37830.github.io/book/00_python/classes.html


In [121]:
import bats
A = [[bats.F2(1), bats.F2(1)], [bats.F2(0), bats.F2(1)]]
Anp = np.array(A)
Anp

array([[1, 1],
       [0, 1]], dtype=object)

In [122]:
Anp @ Anp

array([[1, 0],
       [0, 1]], dtype=object)

In [123]:
2**128

340282366920938463463374607431768211456